In [1]:
# IMPORTS
from tensorflow.python.keras import backend as K
import tensorflow as tf
import pickle
import os
from keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
from time import time
import matplotlib.pyplot as plt
from keras.utils import pad_sequences, to_categorical
from keras.utils.vis_utils import plot_model
from keras.layers import add
from keras.layers import Input, Dense, Dropout, Embedding, LSTM
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tensorflow.keras.applications.xception import Xception

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Setting up the GPU
config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 8} )
sess = tf.compat.v1.Session(config=config) 
K.set_session(sess)

In [ ]:
images_path = '/content/drive/MyDrive/Information Retrieval Project/flickr dataset/flickr30k_images/flickr30k_images'
lst = sorted(os.listdir(images_path))

KeyboardInterrupt: ignored

In [ ]:
train_size = 1600
test_size = 100

train_data = lst[:train_size]
test_data = lst[train_size:train_size+test_size]

In [ ]:
with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/caption_dict.pkl', 'rb') as file:
  content = pickle.load(file)

In [ ]:
train_cap_dict, test_cap_dict = {}, {}
for i in train_data:
    train_cap_dict[i] = content[i]
for i in test_data:
    test_cap_dict[i] = content[i]

In [ ]:
with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/train_cap_dict.pkl', 'wb') as file:
    pickle.dump(train_cap_dict, file)
with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/test_cap_dict.pkl', 'wb') as file:
    pickle.dump(test_cap_dict, file)

In [11]:
# resnet_features = np.load('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/feature vectors/max_pool_train_feature_vectors.npy')
# # /content/drive/MyDrive/Information Retrieval Project/flickr dataset/feature vectors/feature_vectors.npy

word_to_index, index_to_word, caption_dict, resnet_train_encoding, train_content, test_content = None, None, None, None, None, None

with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/word_to_index.pkl', 'rb') as file:
  word_to_index = pickle.load(file)

with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/index_to_word.pkl', 'rb') as file:
  index_to_word = pickle.load(file)

with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/caption_dict.pkl', 'rb') as file:
  caption_dict = pickle.load(file)

# with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/resnet_train_encoding.pkl', 'rb') as file:
#   resnet_train_encoding = pickle.load(file)

# with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/resnet_test_encoding.pkl', 'rb') as file:
#   resnet_test_encoding = pickle.load(file)

with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/train_cap_dict.pkl', 'rb') as file:
  train_cap_dict = pickle.load(file)

with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/test_cap_dict.pkl', 'rb') as file:
  test_cap_dict = pickle.load(file)

VOCAB_SIZE = len(word_to_index) + 1

In [8]:
class xceptionEncoder():
    def __init__(self):
        self.model = Xception(weights = 'imagenet', input_shape = (299, 299, 3))
        self.feature_extractor()

    def feature_extractor(self):
        self.model = Model(self.model.input, self.model.layers[-2].output)

    def get_img_array(self, img):

        img = image.load_img(img, target_size=(299, 299))
        return image.img_to_array(img)
    
    def preprocess_image(self, img):

        img = np.expand_dims(img, axis=0)
        return preprocess_input(img)
    
    def encode_image (self, img):
        img = self.get_img_array(img)
        img = self.preprocess_image(img)
        feature_vector = self.model.predict(img)

        return feature_vector.reshape((-1,))
    
    def get_encoding(self, train_data):
        encoding_dict = {}
        for imageID in train_data:
            image_path = '/content/drive/MyDrive/Information Retrieval Project/flickr dataset/flickr30k_images/flickr30k_images/' + imageID        
            
            encoding_dict[imageID] = self.encode_image(image_path)

        return encoding_dict

In [ ]:
xception = xceptionEncoder()

In [ ]:
xception_train_encoding = xception.get_encoding(train_cap_dict.keys())

In [ ]:
xception_test_encoding = xception.get_encoding(test_cap_dict.keys())

In [ ]:
with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/resnet_train_encoding.pkl', 'wb') as file:
    pickle.dump(resnet_train_encoding, file)
with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/resnet_test_encoding.pkl', 'wb') as file:
    pickle.dump(resnet_test_encoding, file)

In [13]:
with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/xception_train_encoding.pkl', 'wb') as file:
    pickle.dump(xception_train_encoding, file)

In [15]:
with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/xception_test_encoding.pkl', 'wb') as file:
    pickle.dump(xception_test_encoding, file)